# Helpdesk Scheduling

We're trying to come up with a scheduling method for the Helpdesk.

Ruby people might do TDD. I'm going to do NDD (Notebook Driven Development). This document is more or less a stream-of-consciousness of my development process.

First off, we need people to man it.

In [1]:
desk_jockeys = ['Brandon', 'David', 'Brad', 'Kody', 'John', 'Rick', 'Ronan']

We have 2 shifts per day, AM and PM. We need to choose 2 people from the list to do so.

In [2]:
def pick_shifts(jockeys):
    return jockeys[0], jockeys[1]

for x in range(3):
    print(pick_shifts(desk_jockeys))

('Brandon', 'David')
('Brandon', 'David')
('Brandon', 'David')


Well that's not fair. It's always the same 2 guys! We should make sure everyone gets a turn!

In [3]:
def pick_shifts(jockeys):
    j1 = jockeys.pop(0)
    j2 = jockeys.pop(0)
    jockeys.append(j1)
    jockeys.append(j2)
    return j1, j2

for x in range(3):
    print(pick_shifts(desk_jockeys))

('Brandon', 'David')
('Brad', 'Kody')
('John', 'Rick')


Ok, that's a little better. But, now we've got county overlap. Brandon and David are in the same county. They shouldn't both work the same day. Sooo, things are going to get a little complex. By necessity, though. Let's see if we can keep it from becoming [complicated](https://www.python.org/dev/peps/pep-0020/). I'll try and take it by steps to avoid a [How to Draw an Owl](https://i.imgur.com/rCr9A.png) situation.

First off, since we're going to need to keep track of several values in relation to our jockeys, let's go to class.

In [4]:
class DeskJockey():
    def __init__(self, name, county):
        self.name = name
        self.county = county
        self.shifts_worked = 0

    def work(self):
        self.shifts_worked += 1

    def can_work_with(self, coworker):
        if coworker.county == self.county:
            return False
        else:
            return True

    # For those unacquainted with Python, the following 2 methods
    # just change how the objects appear when converted to strings.
    def __str__(self):
        return(self.name)

    def __repr__(self):
        return("<" + self.__str__() + ">")

Now, let's 

In [5]:
brandon = DeskJockey('Brandon', 'Lake')
david = DeskJockey('David', 'Lake')
brad = DeskJockey('Brad', 'Marion')
kody = DeskJockey('Kody', 'Marion')
john = DeskJockey('John', 'Hernando')
rick = DeskJockey('Rick', 'Hernando')
ronan = DeskJockey('Ronan', 'Citrus')

print(brandon.can_work_with(david))
print(david.can_work_with(david))
print(brad.can_work_with(david))

False
False
True


Now, we'll need to sort our list by our jockeys' number of shifts, then try combinations until we find one that works.

In [6]:
desk_jockeys = sorted([brandon, david, brad, kody, john, rick, ronan], key=lambda jockey: jockey.shifts_worked)

def schedule_day(jockeys):
    '''Pick a pair of workers for a day
    Given a list of jockeys, find a pair that can work in a given day.
    
    Does so by looping over the list, one by one, and finding out if there are any eligible partners for that candidate.
    If we reach the end of the list, no one will work that day.
    '''
    to_check = jockeys+[]  # Force this to be a new list
    while to_check:
        first_candidate = to_check.pop(0)
        for second_candidate in jockeys:
            if first_candidate.can_work_with(second_candidate):
                first_candidate.work()
                second_candidate.work()
                desk_jockeys.sort(key=lambda jockey: jockey.shifts_worked)
                return([first_candidate, second_candidate])

    return(None)

for day in range(3):
    print(schedule_day(desk_jockeys))

[<Brandon>, <Brad>]
[<David>, <Kody>]
[<John>, <Ronan>]


Well that could be done cleaner, but it gets us there. Make it work before you make it pretty. Let's test our shift counter.

In [7]:
for jockey in desk_jockeys:
    print(jockey.name, jockey.shifts_worked)

print("**100 days later**")

for day in range(100):
    schedule_day(desk_jockeys)

for jockey in desk_jockeys:
    print(jockey.name, jockey.shifts_worked)

Rick 0
John 1
Ronan 1
David 1
Kody 1
Brandon 1
Brad 1
**100 days later**
John 29
Kody 29
Brandon 29
Brad 29
Rick 30
David 30
Ronan 30


Ok, one last thing. Some of us have earlier/later shifts and need our desk assignment to match. So, let's add that to my class.

In [8]:
class DeskJockeyWithShifts(DeskJockey):
    def __init__(self, name, county, shift="8-5"):
        self.shift = shift
        super(DeskJockeyWithShifts, self).__init__(name, county)

And add that data to our jockeys.

In [9]:
shifts = {'David': 'PM', 'Brandon': 'AM'}

for i, jockey in enumerate(desk_jockeys):
    desk_jockeys[i] = DeskJockeyWithShifts(jockey.name, jockey.county, shifts.get(jockey.name, '8-5'))

And finally update our scheduler to handle shifts.

In [10]:
def schedule_day(jockeys):
    '''Pick a pair of workers for a day
    Given a list of jockeys, find a pair that can work in a given day.
    
    Does so by looping over the list, one by one, and finding out if there are any eligible partners for that candidate.
    If we reach the end of the list, no one will work that day.
    '''
    to_check = jockeys+[]  # Force this to be a new list
    while to_check:
        first_candidate = to_check.pop(0)
        while first_candidate.shift == 'PM':
            if to_check:
                first_candidate = to_check.pop(0)
            else:  # No one left to check
                return(None)
        for second_candidate in jockeys:
            if first_candidate.can_work_with(second_candidate) and second_candidate.shift != 'AM':
                first_candidate.work()
                second_candidate.work()
                desk_jockeys.sort(key=lambda jockey: jockey.shifts_worked)
                return([first_candidate, second_candidate])

    return(None)

print("Shift assignments:")
for jockey in desk_jockeys:
    print("%s: %s" % (jockey.name, jockey.shift))

print("\nDesk schedule:")
for day in range(3):
    print(day, schedule_day(desk_jockeys))

Shift assignments:
John: 8-5
Kody: 8-5
Brandon: AM
Brad: 8-5
Rick: 8-5
David: PM
Ronan: 8-5

Desk schedule:
0 [<John>, <Kody>]
1 [<Brandon>, <Brad>]
2 [<Rick>, <David>]


Looks good to me! Ship it!

> But I need to be able to take days off! Also, weekends?

Oh, right... Ok. Well, let's refactor a bit. I'm going to shortcut weekend handling because I don't feel like writing a bunch of calendar code or going to find a library for it. This can be updated monthly until it seems profitable to refactor.

In [11]:
weekends = {5, 6, 12, 13, 19, 20, 26, 27}

class DeskJockeyWithTimeOff(DeskJockeyWithShifts):
    def __init__(self, name, county, shift='8-5', days_off=weekends):
        self.days_off = days_off
        super(DeskJockeyWithTimeOff, self).__init__(name, county, shift)

    def can_work(self, day, shift, partner=None):
        if day in self.days_off:
            return False
        elif partner and self.county == partner.county:
            return False
        elif self.shift == '8-5' or self.shift == shift:
            return True
        else:
            # Don't know how you got here. :)
            return False

Alright, let's rewrite our jockeys list with our new class. We'll use a [list comprehension](https://docs.python.org/2/tutorial/datastructures.html#list-comprehensions) to recreate our list of jockeys, as it's more Pythonic.

In [12]:
from random import randint  # To simulate random days off

shifts = {'Brandon': 'AM', 'David': 'PM'}

desk_jockeys = [DeskJockeyWithTimeOff(jockey.name,
                                      jockey.county,
                                      shifts.get(jockey.name, '8-5'),
                                      weekends.union({randint(1, 10), randint(1, 10)}))
                for jockey
                in desk_jockeys]

for jockey in desk_jockeys:
    print("%s: %s - %s" % (jockey.name, jockey.shift, jockey.days_off))

Ronan: 8-5 - {1, 5, 6, 9, 12, 13, 19, 20, 26, 27}
Rick: 8-5 - {5, 6, 8, 10, 12, 13, 19, 20, 26, 27}
David: PM - {2, 5, 6, 12, 13, 19, 20, 26, 27}
Brandon: AM - {3, 5, 6, 12, 13, 19, 20, 26, 27}
Brad: 8-5 - {4, 5, 6, 12, 13, 19, 20, 26, 27}
John: 8-5 - {1, 2, 5, 6, 12, 13, 19, 20, 26, 27}
Kody: 8-5 - {5, 6, 8, 12, 13, 19, 20, 26, 27}


And rewrite our scheduler to take days into account.

In [13]:
def schedule_day(jockeys, day):
    '''Pick a pair of workers for a day
    Given a list of jockeys and a day of the month, find a pair that can work on that day.
    
    Does so by looping over the list, one by one, and finding out if there are any eligible partners for that candidate.
    If we reach the end of the list, no one will work that day.
    '''
    to_check = jockeys+[]  # dereference
    while to_check:
        first_candidate = to_check.pop(0)
        while not first_candidate.can_work(day, 'AM'):
            if to_check:
                first_candidate = to_check.pop(0)
            else:  # No one left to check
                return(None)
        for second_candidate in jockeys:
            if first_candidate.can_work(day, 'PM', second_candidate):
                first_candidate.work()
                second_candidate.work()
                desk_jockeys.sort(key=lambda jockey: jockey.shifts_worked)
                return([first_candidate, second_candidate])

    return(None)

print('Schedule:')
for day in range(10):
    print(schedule_day(desk_jockeys, day))

print('\nNumber of shifts worked:')
print(', '.join('%s: %s' % (jockey.name, jockey.shifts_worked) for jockey in desk_jockeys))

Schedule:
[<Ronan>, <Rick>]
[<Brad>, <David>]
[<Kody>, <Brandon>]
[<John>, <Brandon>]
[<John>, <Kody>]
None
None
[<Brad>, <David>]
[<Ronan>, <Rick>]
[<Rick>, <Ronan>]

Number of shifts worked:
David: 2, Brad: 2, John: 2, Kody: 2, Brandon: 2, Ronan: 3, Rick: 3


Well, that's favoring people who just worked. Let's see what I can do about that.

In [14]:
def schedule_day(jockeys, day):
    '''Pick a pair of workers for a day
    Given a list of jockeys and a day of the month, find a pair that can work on that day.
    
    Does so by looping over the list, one by one, and finding out if there are any eligible partners for that candidate.
    If we reach the end of the list, no one will work that day.
    '''
    to_check = jockeys+[]  # dereference
    while to_check:
        first_candidate = to_check.pop(0)
        while not first_candidate.can_work(day, 'AM'):
            if to_check:
                first_candidate = to_check.pop(0)
            else:  # No one left to check
                return(None)
        for second_candidate in jockeys:
            if second_candidate.can_work(day, 'PM', first_candidate):
                first_candidate.work()
                second_candidate.work()
                desk_jockeys.sort(key=lambda jockey: jockey.shifts_worked)
                # Let's try to prevent our candidates getting picked tomorrow
                desk_jockeys.append(desk_jockeys.pop(desk_jockeys.index(first_candidate)))
                desk_jockeys.append(desk_jockeys.pop(desk_jockeys.index(second_candidate)))
                return([first_candidate, second_candidate])

    return(None)

print('Schedule:')
for day in range(10):
    print(schedule_day(desk_jockeys, day))

print('\nNumber of shifts worked:')
print(', '.join('%s: %s' % (jockey.name, jockey.shifts_worked) for jockey in desk_jockeys))

Schedule:
[<Brad>, <David>]
[<Kody>, <Rick>]
[<Brandon>, <Ronan>]
[<John>, <Brad>]
[<Kody>, <David>]
None
None
[<Brandon>, <John>]
[<Ronan>, <Brad>]
[<Rick>, <Kody>]

Number of shifts worked:
David: 4, Brandon: 4, John: 4, Ronan: 5, Brad: 5, Rick: 5, Kody: 5


Alright alright! That's pretty much what we want, I think. Let's just prettify the output a bit.

In [15]:
print('Schedule:\nDay\tAM\tPM')

for day in range(10):
    workpair = schedule_day(desk_jockeys, day)
    if workpair is None:
        print("NONE")
    else:
        print("%s\t%s\t%s" % (day, workpair[0], workpair[1]))

print('\nNumber of shifts worked:')
print(', '.join('%s: %s' % (jockey.name, jockey.shifts_worked) for jockey in desk_jockeys))

Schedule:
Day	AM	PM
0	Brandon	John
1	Brad	David
2	Ronan	Rick
3	Kody	John
4	Brandon	Ronan
NONE
NONE
7	Brad	David
8	John	Ronan
9	Rick	Kody

Number of shifts worked:
Brandon: 6, David: 6, Brad: 7, John: 7, Ronan: 8, Rick: 7, Kody: 7


So, let's clean this up, get a final version. I don't know about you, but I feel like DeskJockeyWithWhatever is a bit wordy. We're not writing Java here!

# tl;dr:

In [16]:
from random import randint

class DeskJockey():
    def __init__(self, name, county, shift='8-5', days_off=weekends):
        self.name = name
        self.county = county
        self.shift = shift
        self.days_off = weekends.union({})  # dereference
        self.shifts_worked = 0

    def can_work(self, day, shift, partner=None):
        if day in self.days_off:
            return False
        elif partner and self.county == partner.county:
            return False
        elif self.shift == '8-5' or self.shift == shift:
            return True
        else:
            # Don't know how you got here. :)
            return False
    
    def take_days_off(self, days):
        self.days_off = self.days_off.union(days)
    
    def work(self):
        self.shifts_worked += 1

    def __str__(self):
        return(self.name)
    
    def __repr__(self):
        return("<" + self.__str__() + ">")
    

def schedule_day(jockeys, day):
    '''Pick a pair of workers for a day
    Given a list of jockeys and a day of the month, find a pair that can work on that day.
    
    Does so by looping over the list, one by one, and finding out if there are any eligible partners for that candidate.
    If we reach the end of the list, no one will work that day.
    '''
    to_check = jockeys+[]  # dereference
    pairing = None
    while to_check and not pairing:
        am_candidate = to_check.pop(0)
        while not am_candidate.can_work(day, 'AM'):
            if to_check:
                am_candidate = to_check.pop(0)
            else:  # No one left to check
                return(None)
        for pm_candidate in jockeys:
            if pm_candidate.can_work(day, 'PM', am_candidate):
                am_candidate.work()
                pm_candidate.work()
                desk_jockeys.sort(key=lambda jockey: jockey.shifts_worked)
                # Let's ensure our candidates don't get picked tomorrow
                desk_jockeys.append(desk_jockeys.pop(desk_jockeys.index(am_candidate)))
                desk_jockeys.append(desk_jockeys.pop(desk_jockeys.index(pm_candidate)))
                return([am_candidate, pm_candidate])

    return(None)

That looks like it'll work! Let's go ahead and create a schedule for April.

In [24]:
# Let's do April
weekends = {2, 3, 9, 10, 16, 17, 23, 24}

desk_jockeys = [DeskJockey(name='Brandon',
                          county='Lake',
                          shift='AM'),
                DeskJockey('David',
                          'Lake',
                          'PM'),
                DeskJockey('Brad',
                          'Marion',
                          'AM'),
                DeskJockey('Kody',
                          'Marion',
                          'PM'),
                DeskJockey('John',
                          'Hernando'),
                DeskJockey('Rick',
                          'Hernando'),
                DeskJockey('Ronan',
                          'Citrus')]

for j in desk_jockeys:
    j.take_days_off({randint(1,30), randint(1,30), randint(1,30)})

print('Start state:')
print('Name\tCounty\tShift\tDays off')
for j in desk_jockeys:
    print('%s\t%s\t%s\t%s' % (j.name, j.county[:7], j.shift, j.days_off))

print('\n\nApril Schedule:')
for day in range(1, 30+1):
    workpair = schedule_day(desk_jockeys, day)
    if workpair is None:
        print("NONE")
    else:
        print("%s\t%s\t%s" % (day, workpair[0], workpair[1]))

print('\nNumber of shifts worked:')
print(', '.join('%s: %s' % (jockey.name, jockey.shifts_worked) for jockey in desk_jockeys))

Start state:
Name	County	Shift	Days off
Brandon	Lake	AM	{2, 3, 6, 9, 10, 15, 16, 17, 21, 23, 24}
David	Lake	PM	{1, 2, 3, 9, 10, 16, 17, 18, 20, 23, 24}
Brad	Marion	AM	{2, 3, 9, 10, 14, 16, 17, 23, 24}
Kody	Marion	PM	{2, 3, 9, 10, 16, 17, 20, 23, 24, 28}
John	Hernand	8-5	{2, 3, 9, 10, 12, 16, 17, 21, 23, 24, 28}
Rick	Hernand	8-5	{2, 3, 9, 10, 16, 17, 22, 23, 24}
Ronan	Citrus	8-5	{2, 3, 9, 10, 15, 16, 17, 23, 24}


April Schedule:
1	Brandon	Kody
NONE
NONE
4	Brad	David
5	John	Ronan
6	Rick	Kody
7	Brandon	John
8	Brad	David
NONE
NONE
11	Ronan	Rick
12	Brandon	Kody
13	John	David
14	Ronan	Rick
15	Brad	John
NONE
NONE
18	Brandon	Kody
19	Ronan	David
20	Rick	Ronan
21	Brad	David
22	John	Kody
NONE
NONE
25	Brandon	Rick
26	Brad	Ronan
27	John	David
28	Brandon	Rick
29	Brad	Ronan
30	John	Kody

Number of shifts worked:
David: 6, Brandon: 6, Rick: 6, Brad: 6, Ronan: 7, John: 7, Kody: 6


That'll do, pig.